### Install Cobrapy Package

In [0]:
!pip install cobra
!pip install pandas --upgrade

# Growth media

The availability of nutrients has a major impact on metabolic fluxes and `cobrapy` provides some helpers to manage the exchanges between the external environment and your metabolic model. In experimental settings the "environment" is usually constituted by the growth medium, ergo the concentrations of all metabolites and co-factors available to the modeled organism. However, constraint-based metabolic models only consider fluxes. Thus, you can not simply use concentrations since fluxes have the unit `mmol / [gDW h]` (concentration per gram dry weight of cells and hour). 

Also, you are setting an upper bound for the particular import flux and not the flux itself. There are some crude approximations. For instance, if you supply 1 mol of glucose every 24h to 1 gram of bacteria you might set the upper exchange flux for glucose to `1 mol / [1 gDW * 24 h]` since that is the nominal maximum that can be imported. There is no guarantee however that glucose will be consumed with that flux. Thus, the preferred data for exchange fluxes are direct flux measurements as the ones obtained from timecourse exa-metabolome measurements for instance. 

So how does that look in COBRApy? The current growth medium of a model is managed by the `medium` attribute.  

In [0]:
import os
from google.colab import files

directory = 'results_media'
if not os.path.exists(directory):
  os.mkdir(directory)

In [0]:
from cobra.test import create_test_model

model = create_test_model("textbook")
biomass_rxn = model.reactions.get_by_id('Biomass_Ecoli_core')
biomass_rxn.id = 'BIOMASS_Ecoli_core_w_GAM' 

In [0]:
model.medium

In [0]:
solution = model.optimize()
print(solution)

solution.fluxes.to_json('results_media/flux_media_1.json')
files.download('results_media/flux_media_1.json')

Load flux solution in [Escher Maps](https://escher.github.io) using the *Core metabolism* map.


This will return a dictionary that contains the upper flux bounds for all active exchange fluxes (the ones having non-zero flux bounds). Right now we see that we have enabled aerobic growth. You can modify a growth medium of a model by assigning a dictionary to `model.medium` that maps exchange reactions to their respective upper import bounds. For now let us enforce anaerobic growth by shutting off the oxygen import.

In [0]:
medium = model.medium
medium["EX_o2_e"] = 0.0
model.medium = medium

model.medium

As we can see oxygen import is now removed from the list of active exchanges and we can verify that this also leads to a lower growth rate.

In [0]:
solution = model.optimize()
print(solution)

solution.fluxes.to_json('results_media/flux_media_2.json')
files.download('results_media/flux_media_2.json')

Load the flux soultion in [Escher Maps](https://escher.github.io) using the *Core metabolism* map.

## Minimal media

In some cases you might be interested in the smallest growth medium that can maintain a specific growth rate, the so called "minimal medium". For this we provide the function `minimal_medium` which by default obtains the medium with the lowest total import flux. This function needs two arguments: the model and the minimum growth rate (or other objective) the model has to achieve.

In [0]:
from cobra.medium import minimal_medium
max_growth = model.optimize().objective_value
minimal_medium(model, max_growth)

So we see that growth is actually limited by glucose import.

Alternatively you might be interested in a minimal medium with the smallest  number of active imports. This can be achieved by using the `minimize_components` argument (note that this uses a MIP formulation and will therefore be much slower).

In [0]:
minimal_medium(model, 0.1, minimize_components=True)

When minimizing the number of import fluxes there may be many alternative solutions. To obtain several of those you can also pass a positive integer to `minimize_components` which will give you at most that many alternative solutions. Let us try that with our model and also use the `open_exchanges` argument which will assign a large upper bound to all import reactions in the model. The return type will be a `pandas.DataFrame`.

In [0]:
minimal_medium(model, 0.1, minimize_components=8, open_exchanges=True)

So there are 4 alternative solutions in total. One aerobic and three anaerobic ones using different carbon sources.